In [8]:
#! pip install linearmodels
#! pip install FixedEffectModel
import pandas as pd
from linearmodels.panel import PanelOLS
from linearmodels.datasets import wage_panel
from fixedeffect.iv import iv2sls, ivgmm, ivtest
from fixedeffect.fe import fixedeffect, did, getfe
from fixedeffect.utils.panel_dgp import gen_data
from sklearn.linear_model import LinearRegression
from statsmodels.stats.diagnostic import het_white
import statsmodels.api as sm

# panel regresion

df1=pd.read_excel("DATE REGRESIE PANEL 2020.xlsx")
df2=pd.read_excel("DATE REGRESIE PANEL 2019.xlsx")
df3=pd.read_excel("DATE REGRESIE PANEL 2018.xlsx")
frames=[df1,df2,df3]
df=pd.concat(frames)
#df.set_index('year', inplace=True)
df.head(10)

,Nationality,year,no_imigrants,gdp_per_cap,unempl_rate,inflation,school_enrl
0,Poland,2020,713,15816.989398,3.160,3.374470,70.477127
1,Romania,2020,347,13047.432062,5.030,2.631073,53.230839
2,India,2020,342,1910.421473,7.997,6.623437,29.441326
3,Ireland,2020,303,85420.190856,5.620,-0.334585,74.680313
4,Italy,2020,215,31911.035789,9.160,-0.137708,69.465279
5,Portugal,2020,168,22242.406418,6.790,-0.012438,70.382088
6,Pakistan,2020,182,1322.315036,4.303,9.739993,0.000000
7,Spain,2020,135,26959.675437,15.530,-0.322753,95.964867
8,France,2020,125,39055.282928,8.010,0.476499,69.346382
9,Lithuania,2020,133,20339.521270,8.490,1.199894,70.786903


In [17]:
# Importing the library
import pandas as pd
import scipy
from scipy import stats
 
df.iloc[:,2:] = df.iloc[:,2:].apply(lambda x: (x-x.mean())/ x.std(), axis=0)
df

,Nationality,year,no_imigrants,gdp_per_cap,unempl_rate,inflation,school_enrl
0,Poland,2020,5.450491,-0.198001,-0.881595,-0.125745,0.571595
1,Romania,2020,2.301849,-0.327034,-0.501284,-0.141385,0.095365
2,India,2020,2.258834,-0.845905,0.102129,-0.057389,-0.561545
3,Ireland,2020,1.923323,3.044802,-0.381293,-0.203781,0.687659
4,Italy,2020,1.166273,0.551819,0.338654,-0.199638,0.543654
...,...,...,...,...,...,...,...
54,Austria,2018,-0.597311,1.462907,-0.537891,-0.154697,1.019243
55,Cyprus,2018,-0.605914,0.435712,0.177988,-0.166539,0.871476
56,Colombia,2018,-0.588709,-0.618938,0.328485,-0.128562,0.153261
57,Estonia,2018,-0.580106,0.144382,-0.432137,-0.124443,0.568511


In [10]:


#OLS (Ordinary Least Squared) Model
# Define the dependent variable and independent variables
y = df["no_imigrants"]
X = df[["gdp_per_cap", "unempl_rate", 'inflation','school_enrl']]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Fit the panel regression model
model = sm.OLS(y, X).fit()

# Print the summary of the regression results
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:           no_imigrants   R-squared:                       0.035
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     1.558
Date:                Sun, 29 Jan 2023   Prob (F-statistic):              0.188
Time:                        23:05:27   Log-Likelihood:                -246.08
No. Observations:                 176   AIC:                             502.2
Df Residuals:                     171   BIC:                             518.0
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -5.898e-17      0.075  -7.87e-16      

In [11]:
# White Test
#perform White's test
white_test = het_white(model.resid,  model.model.exog)WE

#define labels to use for output of White's test
labels = ['Test Statistic', 'Test Statistic p-value', 'F-Statistic', 'F-Test p-value']

#print results of White's test
print(dict(zip(labels, white_test)))

{'Test Statistic': 15.930195264016541, 'Test Statistic p-value': 0.3176557610589532, 'F-Statistic': 1.1444834698108914, 'F-Test p-value': 0.32355978696238064}


In [12]:
#Since the p-value is not less than 0.05, we fail to reject the null hypothesis.
#This means we do not have sufficient evidence to say that heteroscedasticity is present in the regression model.


In [23]:
# Define the dependent variable and independent variables
y = df["no_imigrants"]
X = df[ "unempl_rate", 'inflation','school_enrl',]]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Fit the panel regression model
model2 = sm.OLS(y, X, missing='drop').fit()

# Print the summary of the regression results
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:           no_imigrants   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     2.882
Date:                Sun, 29 Jan 2023   Prob (F-statistic):             0.0914
Time:                        23:23:52   Log-Likelihood:                -247.79
No. Observations:                 176   AIC:                             499.6
Df Residuals:                     174   BIC:                             505.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -2.602e-17      0.075  -3.47e-16      

In [16]:
#Two-Way	Fixed	Effects	Estimator

model3 = PanelOLS(df['no_imigrants',df[["gdp_per_cap", "unempl_rate", 'inflation','school_enrl']],
                  entity_effects=True, time_effects=True).fit
print(model3.summary)

ValueError: Series can only be used with a 2-level MultiIndex